# The following is the first initialization of XGBoost on the TESS astrnomical dataset   

The training data is split 80% 20% training and test. The split is not random with the first 80% of positives being concatenated with the first 80% of training. The same is done for test.  


The model is tested against cross validation using stratified k fold which maintains the original ratio of 1to 0 in the training data when creating folds. The scores for roc_auc are then reported along with the mean and standard deviation. The model is then used to predict the last 20% of the data and the accuracy and roc_auc are reported.   

Moving forward:   

I think that moving forward using grid search is an optimal way of tuning the parameters of the model. As wepreviously discussed I think its a good idea to take the training positive examples (1) and subsample the negative examples to create balanced data sets which can then be used to train the data.   

The accuracy metric is also one of the most important factors in training a good model and I think it is a good idea to research which metrics exist and how they compare to roc_auc as certain papers and competitions have used area under precision recall and kappa as scoring metrics for unbalanced datasets.   

In [1]:
# import modules
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score, StratifiedKFold

# import data and split into 80% training 20% test
train = pd.read_csv('train_TESS_10day_small.csv', header=0)

negative_train = train[203:3293]
positive_train = train[:162]
dtrain = pd.concat([positive_train, negative_train])


negative_test = train[3293:4066]
positive_test = train[162:203]
dtest = pd.concat([negative_test, positive_test])

# retrieve header for features and target
target = 'target'
predictors = [x for x in train.columns if x not in [target]]

# function to fit data and test, takes the model, training and testing data
# the feature headers and number of folds for cross validation
def modelfit(alg, dtrain, dtest, predictors, cv_folds=10):

    # StratifiedKFold for imbalanced data
    cv = StratifiedKFold(n_splits=cv_folds)
    # Test algorithm performance
    scores = cross_val_score(alg, dtrain[predictors], dtrain['target'], cv=cv,
                             scoring='roc_auc')

    # Predict test set:
    alg.fit(dtrain[predictors], dtrain['target'])
    dtest_predictions = alg.predict(dtest[predictors])

    # Print model report:
    print "\nModel Report"
    print " Training Data Cross Validation roc_auc Scores : {0} ".format(scores)
    print " Training Data Mean Score and Std. Dev. {0} {1} ". format(scores.mean(),
                                                                     scores.std())

    print "Test Data Accuracy : {0}".format(metrics.accuracy_score(dtest['target'].values,
                                                                   dtest_predictions))
    print "Test Data roc_auc score : {0}  ".format(metrics.roc_auc_score(dtest['target'],
                                                                         dtest_predictions))
# initialize model and call fitting function
xgb1 = XGBClassifier(
    learning_rate=0.01,
    n_estimators=200,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    nthread=4,
    seed=27)

modelfit(xgb1, dtrain, dtest, predictors)

/home/tom/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)



Model Report
 Training Data Cross Validation roc_auc Scores : [ 0.43061108  0.59889587  0.49747168  0.58394013  0.50738269  0.61963997
  0.46561489  0.43082524  0.40554207  0.52083333] 
 Training Data Mean Score and Std. Dev. 0.506075694841 0.0713935916716 
Test Data Accuracy : 0.949631449631
Test Data roc_auc score : 0.5  
